In [21]:
import sqlite3
import pandas as pd

# Caminho para o banco de dados SQLite
db_path = '../data/processed/ecommerce_olist.db'
conn = sqlite3.connect(db_path)

# Dicionário com as queries
queries = {
    'a': """
        WITH pedidos_por_mes AS (
            SELECT 
                strftime('%Y-%m', order_purchase_timestamp) AS mes,
                COUNT(order_id) AS volume
            FROM pedidos
            GROUP BY mes
        )
        SELECT
            mes,
            volume,
            ROUND(
                (volume - (SELECT AVG(volume) FROM pedidos_por_mes)) * 100.0 / 
                (SELECT AVG(volume) FROM pedidos_por_mes), 
                2
            ) AS variacao_percentual
        FROM pedidos_por_mes
        ORDER BY mes;
    """,
    'b': """
        SELECT
            CASE 
                WHEN JULIANDAY(order_delivered_customer_date) - JULIANDAY(order_approved_at) BETWEEN 0 AND 5 THEN '0-5 dias'
                WHEN JULIANDAY(order_delivered_customer_date) - JULIANDAY(order_approved_at) BETWEEN 6 AND 10 THEN '6-10 dias'
                WHEN JULIANDAY(order_delivered_customer_date) - JULIANDAY(order_approved_at) BETWEEN 11 AND 15 THEN '11-15 dias'
                WHEN JULIANDAY(order_delivered_customer_date) - JULIANDAY(order_approved_at) BETWEEN 16 AND 20 THEN '16-20 dias'
                WHEN JULIANDAY(order_delivered_customer_date) - JULIANDAY(order_approved_at) BETWEEN 21 AND 30 THEN '21-30 dias'
                ELSE 'Mais de 30 dias'
            END AS delivery_time_range,
            COUNT(order_id) AS num_orders
        FROM pedidos
        WHERE order_delivered_customer_date IS NOT NULL
        AND order_approved_at IS NOT NULL
        GROUP BY delivery_time_range
        ORDER BY delivery_time_range;
    """,
    'c': """
        WITH frete_distancia AS (
            SELECT 
                d.distance_km,
                i.freight_value
            FROM pedidos p
            JOIN clientes c ON p.customer_id = c.customer_id
            JOIN itens i ON p.order_id = i.order_id
            JOIN distancias d ON c.customer_zip_code_prefix = d.geolocation_zip_code_prefix
            WHERE d.distance_km IS NOT NULL
        ),
        faixas AS (
            SELECT
                CASE
                    WHEN distance_km < 100 THEN '0-100 km'
                    WHEN distance_km < 200 THEN '100-200 km'
                    WHEN distance_km < 300 THEN '200-300 km'
                    WHEN distance_km < 400 THEN '300-400 km'
                    WHEN distance_km < 500 THEN '400-500 km'
                    WHEN distance_km < 1000 THEN '500-1000 km'
                    WHEN distance_km < 1500 THEN '1000-1500 km'
                    WHEN distance_km < 2000 THEN '1500-2000 km'
                    WHEN distance_km < 2500 THEN '2000-2500 km'
                    ELSE '2500+ km'
                END AS faixa_distancia,
                freight_value
            FROM frete_distancia
        )
        SELECT
            faixa_distancia,
            COUNT(*) AS total_pedidos,
            AVG(freight_value) AS frete_medio
        FROM faixas
        GROUP BY faixa_distancia
        ORDER BY 
            CAST(SUBSTR(faixa_distancia, 1, INSTR(faixa_distancia, '-') - 1) AS INTEGER);
    """,
    'd': """
        SELECT
            pr.product_category_name,
            SUM(it.price * it.order_item_id) AS faturamento
        FROM itens AS it
        JOIN produtos AS pr ON it.product_id = pr.product_id
        GROUP BY pr.product_category_name
        ORDER BY faturamento DESC;
    """,
    'e': """
        SELECT
        c.customer_state,
        SUM(i.price) / COUNT(DISTINCT p.order_id) AS avg_order_value
        FROM pedidos p
        JOIN clientes c ON p.customer_id = c.customer_id
        JOIN itens i ON p.order_id = i.order_id
        GROUP BY c.customer_state
        ORDER BY avg_order_value DESC;
    """
}

# Executando as queries e exibindo os resultados
resultados = {key: pd.read_sql_query(query, conn) for key, query in queries.items()}
conn.close()

# Exibir os 5 primeiros resultados de cada análise
for chave, df in resultados.items():
    print(f"\nResultado da questão {chave}:\n")
    
    if chave == 'a':
        print(df)

        # Lógica para análise da sazonalidade
        picos = df[df['variacao_percentual'] > 20]
        baixas = df[df['variacao_percentual'] < -20]

        print("\n📊 Análise de Sazonalidade:")

        if not picos.empty:
            print(f"- Foram encontrados {len(picos)} meses com variação > +20% (picos de venda):")
            print(picos[['mes', 'variacao_percentual']].to_string(index=False))
        else:
            print("- Nenhum mês com pico de vendas acima de +20%.")

        if not baixas.empty:
            print(f"- Foram encontrados {len(baixas)} meses com variação < -20% (baixas de venda):")
            print(baixas[['mes', 'variacao_percentual']].to_string(index=False))
        else:
            print("- Nenhum mês com baixa de vendas abaixo de -20%.")

        if len(picos) > 0 and len(baixas) > 0:
            print("✅ Existe variação significativa entre os meses. Indício de sazonalidade nas vendas.")
        else:
            print("⚠️ Não há variações suficientes para afirmar sazonalidade com base nesses dados.")
    
    else:
        print(df.head())




Resultado da questão a:

        mes  volume  variacao_percentual
0   2016-09       4               -99.90
1   2016-10     324               -91.85
2   2016-12       1               -99.97
3   2017-01     800               -79.89
4   2017-02    1780               -55.25
5   2017-03    2682               -32.57
6   2017-04    2404               -39.56
7   2017-05    3700                -6.98
8   2017-06    3245               -18.42
9   2017-07    4026                 1.22
10  2017-08    4331                 8.88
11  2017-09    4285                 7.73
12  2017-10    4631                16.43
13  2017-11    7544                89.66
14  2017-12    5673                42.62
15  2018-01    7269                82.75
16  2018-02    6728                69.15
17  2018-03    7211                81.29
18  2018-04    6939                74.45
19  2018-05    6873                72.79
20  2018-06    6167                55.04
21  2018-07    6292                58.18
22  2018-08    6512            